# Problem 3 Sample Code

This sample code is meant as a guide on how to use PyTorch and how to use the relevant model layers. This not a guide on how to design a network and the network in this example is intentionally designed to have poor performace.

In [66]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import TensorDataset, Dataset, DataLoader
import seaborn as sns
from sklearn.model_selection import train_test_split

In [67]:
df = pd.read_csv('out.csv')

df = df.replace('',np.nan)
df = df.dropna(axis="rows", how="any")
df = df.drop(columns= "uid")

df_X = df.drop(columns= "label")
for col in df_X:
    df_X[col] = [(element - np.mean(df_X[col])) / np.std(df_X[col]) for element in df_X[col]]
print("===dataframe=== \n {}".format(df))
X = np.array(df_X)
y = np.array(df["label"])

# def normalize(x):
#     x = np.asarray(x)
#     return (x - x.min()) / (np.ptp(x))
# X_normed = []
# for i in range(0, len(X)):
#     X_normed.append(X[i])
# # X = (X - X.mean())/(X.std())
# print(" - NORMALIZED  X \n {} - ".format(X))


train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)
# print(" pre TRAIN X \n {}".format(train_X))

train_X = torch.tensor(train_X, dtype=torch.float32)
# print("TENSOR  X \n {}".format(train_X))
train_y = torch.tensor(train_y, dtype=torch.long)
# 80/20 x values as floats
test_X = torch.tensor(test_X, dtype=torch.float32)
test_y = torch.tensor(test_y, dtype=torch.long)
print("test_x torch tensor {}".format(test_X))
print("test_x torch tensor shape {}".format(test_X.shape))

train_dataset = TensorDataset(train_X, train_y)
test_dataset = TensorDataset(test_X, test_y)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True) 

print ("x \n{}".format(df))
print ("y \n{}".format(df))

# x_train = torch.tensor(train[['chi','lin_reg','lin_reg_chi','stddev_step_speed']].copy().to_numpy(), dtype=torch.float32)
# y_train = torch.tensor(train['label'].copy().to_numpy(), dtype=torch.long)
# x_test = torch.tensor(train[['chi','lin_reg','lin_reg_chi','stddev_step_speed']].copy().to_numpy(), dtype=torch.float32)
# y_test = torch.tensor(train['label'].copy().to_numpy(), dtype=torch.long)
# train_dataset = TensorDataset(x_train, y_train)
# test_dataset = TensorDataset(x_test, y_test)

===dataframe=== 
        label           chi   lin_reg  lin_reg_chi  stddev_step_speed
0          0  8.474767e+04  0.992533     0.988869          17.360221
1          0  2.097220e+04  0.973844     1.938908          17.308306
2          0  2.356781e+04  0.894242     6.604954          17.300169
3          0  8.100880e+03  0.967026     5.758685          17.376369
4          0  5.593839e+05  0.000296     1.512429          17.205146
...      ...           ...       ...          ...                ...
16075      0  4.669320e+05  0.999245     0.867075          18.036484
16076      0  1.375967e+06  0.999707     1.345129          26.151765
16077      0  2.235552e+06  0.998892     3.587700          23.801682
16078      0  6.701228e+05  0.998953     0.680709          16.319528
16079      0  6.096492e+05  0.999056     7.286809          30.031311

[16013 rows x 5 columns]
test_x torch tensor tensor([[-0.2651,  0.5888, -0.4655,  0.6117],
        [-0.4146,  0.5887, -0.4658,  0.5065],
        [-0.4055

We can convert images to numpy arrays and plot them with matplotlib:

## Network Definition
Let's instantiate a model and take a look at the layers.

In [68]:
model = nn.Sequential(
    nn.Linear(4, 10),
    nn.ReLU(),
    nn.Linear(10, 5),
    nn.ReLU(),
    nn.Linear(5, 2),
    nn.Sigmoid()
)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# loss_fn = nn.BCELoss()
loss_fn = nn.CrossEntropyLoss()

torch.manual_seed(15552494823729223621)

## Training
We also choose an optimizer and a loss function.

We could write our training procedure manually and directly index the `Dataset` objects, but the `DataLoader` object conveniently creates an iterable for automatically creating random minibatches:

We now write our backpropagation loop, training for 10 epochs.

In [69]:
model.train()

for epoch in range(10):
    for batch_idx, (data, target) in enumerate(train_loader):
        # print("data \n{}".format(data))
        # Erase accumulated gradients
        optimizer.zero_grad()

        # Forward pass
        output = model(data)
        # print(" -- OPUTPUT -- \n{}".format(output))
        # output = output.flatten()

        # Calculate loss
        loss = loss_fn(output, target)

        # Backward pass
        loss.backward()
        
        # Weight update
        optimizer.step()

    # Track loss each epoch
    print('Train Epoch: %d  Loss: %.4f' % (epoch + 1,  loss.item()))

Train Epoch: 1  Loss: 0.6908
Train Epoch: 2  Loss: 0.6769
Train Epoch: 3  Loss: 0.6777
Train Epoch: 4  Loss: 0.5901
Train Epoch: 5  Loss: 0.5615
Train Epoch: 6  Loss: 0.5698
Train Epoch: 7  Loss: 0.4790
Train Epoch: 8  Loss: 0.4798
Train Epoch: 9  Loss: 0.4418
Train Epoch: 10  Loss: 0.3738


## Testing
We can perform forward passes through the network without saving gradients.

In [70]:
# Putting layers like Dropout into evaluation mode
model.eval()

test_loss = 0
correct = 0

# Turning off automatic differentiation
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        print(output)
        test_loss += loss_fn(output, target).item()  # Sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max class score
        print(pred)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('Test set: Average loss: %.4f, Accuracy : %d/%d (%.4f)' %
      (test_loss, correct, len(test_loader.dataset),
       100. * correct / len(test_loader.dataset)))

tensor([[0.1341, 0.9826],
        [0.9315, 0.1989],
        [0.9073, 0.3236],
        [0.9367, 0.1971],
        [0.8107, 0.4791],
        [0.1246, 0.9859],
        [0.9641, 0.1879],
        [0.9467, 0.2077],
        [0.0016, 1.0000],
        [0.2225, 0.9418],
        [0.9354, 0.2027],
        [0.9385, 0.1885],
        [0.9350, 0.2022],
        [0.1267, 0.9847],
        [0.4279, 0.8474],
        [0.9091, 0.2232],
        [0.0769, 0.9971],
        [0.2094, 0.9398],
        [0.9384, 0.1884],
        [0.0775, 0.9972],
        [0.9094, 0.2229],
        [0.0518, 0.9990],
        [0.0353, 0.9997],
        [0.9422, 0.1872],
        [0.1955, 0.9656],
        [0.0739, 0.9974],
        [0.0258, 0.9999],
        [0.2595, 0.8908],
        [0.9323, 0.1996],
        [0.0255, 0.9999],
        [0.9637, 0.1861],
        [0.9324, 0.2066]])
tensor([[1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0]

In [71]:
#   REAL TESTING
test = pd.read_csv('test.csv')

df_vals = test.drop(columns= "uid")
for col in df_vals:
    df_vals[col] = [(element - np.mean(df_vals[col])) / np.std(df_vals[col]) for element in df_vals[col]]
print("===dataframe=== \n {}".format(test))

test_X = np.array(df_vals)
test_X = torch.tensor(test_X, dtype=torch.float32)

print(test_X)
print(test_X.shape)



===dataframe=== 
            uid           chi   lin_reg  lin_reg_chi  stddev_step_speed
0     lab_19_0  1.226808e+04  0.878979     0.267350          19.661653
1     lab_19_1  2.027183e+05  0.072402   407.576553          18.135286
2     lab_19_2  1.720428e+07  0.998963    27.825655          86.806286
3     lab_19_3  1.170899e+07  0.991160   170.673846          81.682927
4     lab_19_4  1.947901e+07  0.987416     1.985455          10.228939
..         ...           ...       ...          ...                ...
472   lab_42_9  3.893311e+06  0.989693   549.380346          85.433759
473  lab_42_10  9.995795e+04  0.926821   244.652959          32.603626
474  lab_42_11  1.644216e+06  0.952655   132.548405          17.927726
475  lab_42_12  1.712307e+06  0.999726     1.653985          36.955453
476  lab_42_13  2.899105e+05  0.997562    18.285068          46.364285

[477 rows x 5 columns]
tensor([[-0.2431, -0.0117, -0.2071, -0.6031],
        [-0.2309, -3.3169,  0.2097, -0.6731],
        [ 0.85

In [72]:
output = model(test_X)
#print(output)
pred = output.argmax(dim=1, keepdim=True)

pred = np.array(pred)
print(pred)
submission = pd.DataFrame(columns=['UID', 'label'])
submission['UID'] = test['uid']
submission['label'] = pred
print(submission)

submission.to_csv('submission.csv', index=False)

[[1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
